## DB와 연결하려 했으나 실패

In [9]:
import psycopg2

db_connect = psycopg2.connect(
        user="tasha",
        password="tasha",
        host="192.168.253.128",
        port=5432,
        database="tasha-progress-server",
        connect_timeout=3
)

OperationalError: connection to server at "192.168.253.128", port 5432 failed: timeout expired


## import & api test

In [10]:
import requests
import json
import pandas as pd
from tqdm import tqdm
import itertools
import pickle
import numpy as np

In [11]:
with open('./username_sample.pkl', 'rb') as f:
	username_list = pickle.load(f)

In [29]:
url = 'http://ws.audioscrobbler.com/2.0'

params = {
    "method": "user.getrecenttracks",
    "extended": 1,
    "user": "JM",
    "limit": 1,
    "api_key": "1a62f2d4937452d62d7426029e4c9997",
    "format": "json",
}

text = requests.get(url, params).text
texts = pd.DataFrame(json.loads(text)['recenttracks']['track'])

display(texts)
print(text)

,artist,date,mbid,name,image,url,streamable,album,loved
0,{'url': 'https://www.last.fm/music/my+bloody+v...,"{'uts': '1108290000', '#text': '13 Feb 2005, 1...",04e30386-e709-4a86-adc7-ca2ff8ed729a,Sometimes,"[{'size': 'small', '#text': ''}, {'size': 'med...",https://www.last.fm/music/my+bloody+valentine/...,0,"{'mbid': '', '#text': ''}",1


{"recenttracks":{"track":[{"artist":{"url":"https:\/\/www.last.fm\/music\/my+bloody+valentine","name":"my bloody valentine","image":[{"size":"small","#text":"https:\/\/lastfm.freetls.fastly.net\/i\/u\/34s\/2a96cbd8b46e442fc41c2b86b821562f.png"},{"size":"medium","#text":"https:\/\/lastfm.freetls.fastly.net\/i\/u\/64s\/2a96cbd8b46e442fc41c2b86b821562f.png"},{"size":"large","#text":"https:\/\/lastfm.freetls.fastly.net\/i\/u\/174s\/2a96cbd8b46e442fc41c2b86b821562f.png"},{"size":"extralarge","#text":"https:\/\/lastfm.freetls.fastly.net\/i\/u\/300x300\/2a96cbd8b46e442fc41c2b86b821562f.png"}],"mbid":""},"date":{"uts":"1108290000","#text":"13 Feb 2005, 10:20"},"mbid":"04e30386-e709-4a86-adc7-ca2ff8ed729a","name":"Sometimes","image":[{"size":"small","#text":""},{"size":"medium","#text":""},{"size":"large","#text":""},{"size":"extralarge","#text":""}],"url":"https:\/\/www.last.fm\/music\/my+bloody+valentine\/_\/Sometimes","streamable":"0","album":{"mbid":"","#text":""},"loved":"1"}],"@attr":{"us

In [64]:
import requests
import json
import pandas as pd
from tqdm import tqdm
import itertools

url = 'http://ws.audioscrobbler.com/2.0'

params = {
    "method": "user.getrecenttracks",
    "extended": 1,
    "user": "",
    "limit": 1,
    "api_key": "1a62f2d4937452d62d7426029e4c9997",
    "format": "json",
}

dataframe_list = []
for i, user in enumerate(tqdm(username_list)):
    params["user"] = user
    params["limit"] = 1
    text = requests.get(url, params).text
    text = json.loads(text)
    text = text['recenttracks']
    track = text['track']
    attr = text['@attr']
    params["limit"] = 200
    for page in range(int(attr['totalPages'])):
        params["page"] = page + 1
        text = requests.get(url, params).text
        texts = pd.DataFrame(json.loads(text)['recenttracks']['track'])
        texts['username'] = user
        texts['user_total'] = attr['total']

        texts['album_mbid'] = texts['album'].apply(lambda x: x['mbid'])
        texts['album_name'] = texts['album'].apply(lambda x: x['#text'])

        # texts['date_uts'] = texts['date'].apply(lambda x: np.NAN if x==np.NAN else x['uts'])
        texts['date_uts'] = texts['date'].apply(lambda x: x['uts'])
        texts['date_string'] = texts['date'].apply(lambda x: x['#text'])

        texts['artist_url'] = texts['artist'].apply(lambda x: x['url'])
        texts['artist_name'] = texts['artist'].apply(lambda x: x['name'])
        texts['artist_mbid'] = texts['artist'].apply(lambda x: x['mbid'])

        texts.drop(columns=['album', 'date', 'artist', 'image'], inplace=True)
        dataframe_list.append(texts)
        break
data_csv = pd.concat(dataframe_list, ignore_index=True)
data_csv.rename({'mbid':'track_mbid', 'name':'track_name', 'url':'track_url'})
data_csv.replace('', np.NaN, inplace=True)
display(data_csv)

 11%|█         | 11/100 [00:12<01:42,  1.15s/it]


,mbid,name,url,streamable,loved,username,user_total,album_mbid,album_name,date_uts,date_string,artist_url,artist_name,artist_mbid
0,NaN,"Requiem in D Minor, K. 626: 4. Offertorium: Do...",https://www.last.fm/music/Wolfgang+Amadeus+Moz...,0,0,cynocauma,29607,NaN,Mozart: Requiem,1674264534,"21 Jan 2023, 01:28",https://www.last.fm/music/Wolfgang+Amadeus+Mozart,Wolfgang Amadeus Mozart,NaN
1,NaN,"Requiem in D Minor, K. 626: 3. Sequentia: Lacr...",https://www.last.fm/music/Wolfgang+Amadeus+Moz...,0,0,cynocauma,29607,NaN,Mozart: Requiem,1674264364,"21 Jan 2023, 01:26",https://www.last.fm/music/Wolfgang+Amadeus+Mozart,Wolfgang Amadeus Mozart,NaN
2,NaN,"Requiem in D Minor, K. 626: 3. Sequentia: Conf...",https://www.last.fm/music/Wolfgang+Amadeus+Moz...,0,0,cynocauma,29607,NaN,Mozart: Requiem,1674264207,"21 Jan 2023, 01:23",https://www.last.fm/music/Wolfgang+Amadeus+Mozart,Wolfgang Amadeus Mozart,NaN
3,NaN,"Requiem in D Minor, K. 626: 3. Sequentia: Reco...",https://www.last.fm/music/Wolfgang+Amadeus+Moz...,0,0,cynocauma,29607,NaN,Mozart: Requiem,1674263903,"21 Jan 2023, 01:18",https://www.last.fm/music/Wolfgang+Amadeus+Mozart,Wolfgang Amadeus Mozart,NaN
4,NaN,"Requiem in D Minor, K. 626: 3. Sequentia: Rex ...",https://www.last.fm/music/Wolfgang+Amadeus+Moz...,0,0,cynocauma,29607,NaN,Mozart: Requiem,1674263798,"21 Jan 2023, 01:16",https://www.last.fm/music/Wolfgang+Amadeus+Mozart,Wolfgang Amadeus Mozart,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,NaN,Sur le drapeau,https://www.last.fm/music/Supr%C3%AAme+NTM/_/S...,0,0,cynocauma,29607,NaN,Sur le drapeau (Extrait du projet 93 Empire),1674156906,"19 Jan 2023, 19:35",https://www.last.fm/music/Supr%C3%AAme+NTM,Suprême NTM,NaN
2396,NaN,Sur le drapeau,https://www.last.fm/music/Supr%C3%AAme+NTM/_/S...,0,0,cynocauma,29607,NaN,Sur le drapeau (Extrait du projet 93 Empire),1674156707,"19 Jan 2023, 19:31",https://www.last.fm/music/Supr%C3%AAme+NTM,Suprême NTM,NaN
2397,NaN,Sur le drapeau,https://www.last.fm/music/Supr%C3%AAme+NTM/_/S...,0,0,cynocauma,29607,NaN,Sur le drapeau (Extrait du projet 93 Empire),1674156507,"19 Jan 2023, 19:28",https://www.last.fm/music/Supr%C3%AAme+NTM,Suprême NTM,NaN
2398,NaN,Sur le drapeau,https://www.last.fm/music/Supr%C3%AAme+NTM/_/S...,0,0,cynocauma,29607,NaN,Sur le drapeau (Extrait du projet 93 Empire),1674156308,"19 Jan 2023, 19:25",https://www.last.fm/music/Supr%C3%AAme+NTM,Suprême NTM,NaN


## mbid 에 null 이 너무 많음 & user 끼리 중복됨

In [66]:
data_csv.notna().sum()
# mbid 에는 null 이 많아서 name 으로 대체

mbid           1307
name           2400
url            2400
streamable     2400
loved          2400
username       2400
user_total     2400
album_mbid     1396
album_name     2365
date_uts       2400
date_string    2400
artist_url     2400
artist_name    2400
artist_mbid       0
dtype: int64

In [23]:
from collections import Counter

result = Counter(username_list)
print(result)

for key in result:
    print(key, result[key])

result = Counter(username_list).values()
print(result)

Counter({'erickdsr12': 14, 'cynocauma': 13, 'NovicePlayer': 12, 'leyrey': 10, 'Nouwh': 10, 'Spectrum321146': 10, 'ZombieTeam67': 9, 'BilingualZebra7': 9, 'AetherJoako': 8, 'rataclans': 5})
cynocauma 13
AetherJoako 8
leyrey 10
NovicePlayer 12
Nouwh 10
erickdsr12 14
ZombieTeam67 9
BilingualZebra7 9
rataclans 5
Spectrum321146 10
dict_values([13, 8, 10, 12, 10, 14, 9, 9, 5, 10])


## iteraction 수집 & 각종 dictionary 들 선언

In [36]:
import requests
import json
import pandas as pd
from tqdm import tqdm
import itertools

url = 'http://ws.audioscrobbler.com/2.0'

params_inter = {
    "method": "user.getrecenttracks",
    "extended": 1,
    "user": "",
    "limit": 1,
    "api_key": "1a62f2d4937452d62d7426029e4c9997",
    "format": "json",
}

user2id = {}
track2id = {}
album2id = {}
artist2id = {}

track2artist = {}
artist2album = {}

album2track = {}

def push_track2id(str):
    if str not in track2id:
        track2id[str] = len(track2id)

def push_user2id(str):
    if str not in user2id:
        user2id[str] = len(user2id)

def push_album2id(str):
    if str not in album2id:
        album2id[str] = len(album2id)

def push_artist2id(str):
    if str not in artist2id:
        artist2id[str] = len(artist2id)

def push_track2artist(track, artist):
    if track not in track2artist:
        track2artist[track] = artist

def push_artist2album(artist, album):
    if isinstance(album, float) or album == '':
        return
    if artist not in artist2album:
        artist2album[artist] = [album]
    elif album not in artist2album[artist]:
        artist2album[artist].append(album)

def push_album2track(album, track):
    if album not in album2track:
        album2track[album] = [track]
    elif album not in album2track[album]:
        album2track[album].append(album)

for user in username_list:
    push_user2id(user)

# id2item, id2user dict 는 만들기가 까다로움. 업데이트가 까다롭다.

dataframe_list = []
for user, i in tqdm(user2id.items()):
    params_inter["user"] = user
    params_inter["limit"] = 1
    text = requests.get(url, params_inter).text
    text = json.loads(text)
    text = text['recenttracks']
    track = text['track']
    attr = text['@attr']
    params_inter["limit"] = 200
    for page in range(int(attr['totalPages'])):
        params_inter["page"] = page + 1
        text = requests.get(url, params_inter).text
        texts = json.loads(text)['recenttracks']['track']
        texts = pd.DataFrame(texts)
        texts['user_name'] = user
        # texts['user_total'] = attr['total']
        texts['album_name'] = texts['album'].apply(lambda x: x['#text'])
        texts['timestamp_uts'] = texts['date'].apply(lambda x: float('nan') if isinstance(x, float) else x['uts'])
        # texts['date_string'] = texts['date'].apply(lambda x: float('nan') if isinstance(x, float) else x['#text'])
        texts['artist_name'] = texts['artist'].apply(lambda x: x['name'])
        texts['track_name'] = texts['name']

        texts['track_name'].apply(lambda x: push_track2id(x))
        texts['album_name'].apply(lambda x: push_album2id(x))
        texts['artist_name'].apply(lambda x: push_artist2id(x))
        texts.apply(lambda x: push_track2artist(x['name'], x['artist_name']), axis=1)
        texts.apply(lambda x: push_artist2album(x['artist_name'], x['album_name']), axis=1)

        texts.drop(columns=['album', 'date', 'artist', 'image', 'mbid', 'url', 'streamable', 'name'], inplace=True)
        if '@attr' in list(texts.keys()):
            texts.drop(columns=['@attr'], inplace=True)
        
        dataframe_list.append(texts)
        break

data_csv = pd.concat(dataframe_list, ignore_index=True)
data_csv.rename(columns={'name':'track_name'}, inplace=True)
data_csv.replace('', np.NaN, inplace=True)
data_csv = data_csv.astype({
    'track_name':'string',
    'loved':'int8',
    'user_name':'string',
    'album_name':'string',
    'timestamp_uts':'int64',
    'artist_name':'string',
})
display(data_csv)
data_csv.to_csv("interaction_2000.csv", index=False)

100%|██████████| 10/10 [00:13<00:00,  1.39s/it]


,loved,user_name,album_name,timestamp_uts,artist_name,track_name
0,0,cynocauma,The Search,1674687774,NF,The Search
1,0,cynocauma,The Search,1674687526,NF,The Search
2,0,cynocauma,The Search,1674687277,NF,The Search
3,0,cynocauma,The Search,1674687029,NF,The Search
4,0,cynocauma,The Search,1674686780,NF,The Search
...,...,...,...,...,...,...
1995,0,Spectrum321146,Groove,1674490044,StreamBeats by Harris Heller,Stopgap
1996,0,Spectrum321146,Groove,1674489923,StreamBeats by Harris Heller,Disco Ball
1997,0,Spectrum321146,Groove,1674489783,StreamBeats by Harris Heller,Groove All Night
1998,0,Spectrum321146,Groove,1674489665,StreamBeats by Harris Heller,Rehearsal


In [5]:
data_csv.dtypes

track_name     string
loved            int8
username       string
album_name     string
date_uts       string
date_string    string
artist_name    string
dtype: object

In [4]:
data_csv[data_csv.album_name.isnull()].head()

,track_name,loved,username,album_name,date_uts,date_string,artist_name
439,Летучая мышь,0,leyrey,NaN,1674500543,"23 Jan 2023, 19:02",Филипп Киркоров
523,Opening 2 Full,0,leyrey,NaN,1674473847,"23 Jan 2023, 11:37",Death Note
561,I Need Air,0,leyrey,NaN,1674429812,"22 Jan 2023, 23:23",Nothing But Thieves
569,"Man [Blue Madonna, 2018]",0,leyrey,NaN,1674428025,"22 Jan 2023, 22:53",BØRNS
576,Звезда по имени солнце,0,leyrey,NaN,1674426397,"22 Jan 2023, 22:26","В. Цой, Кино"


In [103]:
print(len(artist2album)) # album 이 없는 아티스트도 존재하는 듯
print(len(artist2id))

816
838


In [91]:
# temp_list = [user2id, track2id, album2id, artist2id, track2artist]

# with open('dictionary_list_0122.pkl', 'wb') as f:
# 	pickle.dump(temp_list, f, protocol=pickle.HIGHEST_PROTOCOL)

## track_info

In [41]:
params_track = {
    "method": "track.getInfo",
    "track": "",
    "artist": "",
    "api_key": "1a62f2d4937452d62d7426029e4c9997",
    "autocorrect": 0,
    "format": "json",
}

tag2id = {}

def push_tag2id(tags):
    for tag in tags:
        if tag not in tag2id:
            tag2id[tag] = len(tag2id)

trackInfo_dataframe_list = []
for i, (track, artist) in enumerate(tqdm(track2artist.items())):
    params_track['track'] = track
    params_track['artist'] = artist
    track = requests.get(url, params_track).text
    track = json.loads(track)['track']

    track['artist_name'] = track['artist']['name']
    # if 'mbid' in list(track['artist'].keys()):
    #     track['artist_mbid'] = track['artist'].apply(lambda x: x['mbid'])
    # else:
    #     track['artist_mbid'] = np.nan
    track['artist_url'] = track['artist']['url']

    # track['tag'] = track['toptags'].apply(lambda x: x['tag']) # 태그가 다 없음
    track['tags'] = list(pd.DataFrame(track['toptags']['tag']).apply(lambda x: (x['name']), axis=1))
    # print(track['tag'])
    push_tag2id(track['tags'])

    # track['toptags'].apply(lambda x: push_tag2id(x['tag']), axis=1)
    # track['tag'] = track['toptags'].apply(lambda x: x['tag'] if x != list([]) else np.nan)
    
    if 'album' in list(track.keys()):
        track['album_name'] = track['album']['title']
    #     track['album_url'] = track['album'].apply(lambda x: x['image'][-1]['#text'])
    #     track['album_artist'] = track['album'].apply(lambda x: x['artist'])
    #     track.drop(columns=['album'], inplace=True)
    else:
        track['album_name'] = np.nan
    #     track['album_url'] = np.nan
    #     track['album_artist'] = np.nan

    track = pd.DataFrame([track])
    if 'album' in list(track.keys()):
        track.drop(columns=['album'], inplace=True)
    
    if 'wiki' in list(track.keys()):
        track.drop(columns=['wiki'], inplace=True)

    if 'mbid' in list(track.keys()):
        track.drop(columns=['mbid'], inplace=True)
    track['streamable_text'] = track['streamable'].apply(lambda x: x['#text'])
    track['streamable_fulltrack'] = track['streamable'].apply(lambda x: x['fulltrack'])
    track.drop(columns=['artist', 'streamable', 'toptags'], inplace=True)
    trackInfo_dataframe_list.append(track)

trackInfo_csv = pd.concat(trackInfo_dataframe_list, ignore_index=True)
trackInfo_csv.rename(columns={'name':'track_name', 'tags':'track_tag_list'}, inplace=True)
trackInfo_csv = trackInfo_csv.astype({
    'track_name':'string',
    # 'url':'string',
    'duration':'int32',
    'listeners':'int32',
    'playcount':'int32',
    'artist_name':'string',
    'artist_url':'string',
    'track_tag_list':'object',
    'artist_url':'string',
    'streamable_text':'int8',
    'streamable_fulltrack':'int8'
})
display(trackInfo_csv.head())
trackInfo_csv.to_csv("trackinfo_50.csv", index=False)

  1%|          | 11/1270 [00:02<05:01,  4.17it/s]


,track_name,url,duration,listeners,playcount,artist_name,artist_url,track_tag_list,album_name,streamable_text,streamable_fulltrack
0,The Search,https://www.last.fm/music/NF/_/The+Search,248000,176807,2074521,NF,https://www.last.fm/music/NF,"[Hip-Hop, rap, conscious hip-hop, trap]",The Search,0,0
1,Le Boeing,https://www.last.fm/music/Oscar+les+vacances/_...,187000,31,145,Oscar les vacances,https://www.last.fm/music/Oscar+les+vacances,[],Vroum,0,0
2,Le rendez-vous au skatepark,https://www.last.fm/music/Oscar+les+vacances/_...,173000,32,162,Oscar les vacances,https://www.last.fm/music/Oscar+les+vacances,[],Vroum,0,0
3,Superman,https://www.last.fm/music/Oscar+les+vacances/_...,160000,57,318,Oscar les vacances,https://www.last.fm/music/Oscar+les+vacances,[],Vroum,0,0
4,Vroum vroum,https://www.last.fm/music/Oscar+les+vacances/_...,162000,95,364,Oscar les vacances,https://www.last.fm/music/Oscar+les+vacances,[],Vroum - EP,0,0


In [12]:
trackInfo_csv.dtypes

track_name              string
url                     string
duration                 int32
listeners                int32
playcount                int32
artist_name             string
artist_url              string
track_tag_list          object
album_title             string
streamable_text           int8
streamable_fulltrack      int8
dtype: object

In [6]:
trackInfo_csv.notna().sum() # 수정 후: mbid, title, album, wiki, artist_mbid 는 삭제

track_name              52
url                     52
duration                52
listeners               52
playcount               52
artist_name             52
artist_url              52
track_tag_list          52
album_title             48
streamable_text         52
streamable_fulltrack    52
dtype: int64

In [7]:
(trackInfo_csv['streamable_fulltrack'] == trackInfo_csv['streamable_text']).sum()

52

## album info

In [42]:
params_album = {
    "method": "album.getInfo",
    "album": "",
    "artist" : "",
    "api_key": "1a62f2d4937452d62d7426029e4c9997",
    "autocorrect": 0,
    "format": "json",
}

albumInfo_dataframe_list = []
for i, (artist, albums) in enumerate(tqdm(artist2album.items())):
    params_album['artist'] = artist
    for album in albums:
        params_album['album'] = album
        album = requests.get(url, params_album).text
        try:
            album = json.loads(album)['album']
        except:
            print(album)
            print(artist, albums, album, params_album)
        
        album['tag'] = album['tags']
        # print(album['image'][-1])
        album['image_url'] = album['image'][-1]['#text']
        # print(album['wiki'])

        try:
            album['published'] = album['wiki']['published']
            # album['summary'] = album['wiki']['summary']
            # album['content'] = album['wiki']['content']
            album = pd.DataFrame([album])
            album.drop(columns = ['wiki'], inplace=True)
        except:
            # album['summary'] = np.nan
            # album['content'] = np.nan
            album = pd.DataFrame([album])

        # display(pd.DataFrame(album['tracks'].item()['track']))
        # album['track'] = list(album['tracks']['track'].apply(lambda x: (x['name'], x['artist']['name'])))
        # print(pd.DataFrame(album['tracks'].item()['track']))
        # print(album['tags'][0])

        # try:
        #     if isinstance(album['tracks'].item()['track'], list):
        #         album['track'] = [list(pd.DataFrame(album['tracks'].item()['track']).apply(lambda x: (x['name'], x['artist']['name']), axis=1))]
        #     else:
        #         album['track'] = [list(pd.DataFrame([album['tracks'].item()['track']]).apply(lambda x: (x['name'], x['artist']['name']), axis=1))]
        # except:
        #     album['track'] = np.nan # album 을 조회했는데 tracks 가 없는 경우..?
        
        try:
            album['tag'] = [list(pd.DataFrame(album['tags'].item()['tag']).apply(lambda x: (x['name']), axis=1))]
        except:
            album['tag'] = np.nan
            break
        albumInfo_dataframe_list.append(album)
        if 'tracks' in list(album.keys()):
            album.drop(columns=['tracks'], inplace=True)

        album.drop(columns=['tags', 'mbid', 'image'], inplace=True)

albumInfo_csv = pd.concat(albumInfo_dataframe_list, ignore_index=True)
albumInfo_csv.rename(columns={'artist':'artist_name', 'name':'album_name'}, inplace=True)
albumInfo_csv.replace('', np.NaN, inplace=True)
albumInfo_csv = albumInfo_csv.astype({
    'artist_name':'string',
    'album_name':'string',
    'image_url':'string',
    'listeners':'int32',
    'playcount':'int32',
    'url':'string',
    'tag':'object',
    'published':'string',
})
display(albumInfo_csv)
albumInfo_csv.to_csv("albumInfo_50.csv", index=False)

  4%|▍         | 31/741 [00:15<06:00,  1.97it/s]


,artist_name,playcount,url,album_name,listeners,tag,image_url,published
0,NF,12490521,https://www.last.fm/music/NF/The+Search,The Search,322774,"[rap, hip-hop, 2019, 2010s, emotional rap]",https://lastfm.freetls.fastly.net/i/u/300x300/...,"03 Mar 2022, 14:41"
1,NF,9924710,https://www.last.fm/music/NF/Perception,Perception,422252,"[nf, best of 2017, 2017, rap, american]",https://lastfm.freetls.fastly.net/i/u/300x300/...,"10 Oct 2022, 04:15"
2,JVG,75692,https://www.last.fm/music/JVG/Ikuinen+vappu,Ikuinen vappu,7389,"[suomirap, 2010s, 2019, pop, finnish]",https://lastfm.freetls.fastly.net/i/u/300x300/...,<NA>
3,Wolfgang Amadeus Mozart,1670653,https://www.last.fm/music/Wolfgang+Amadeus+Moz...,Mozart: Requiem,133983,"[classical, requiem, mozart, choral, conductor]",https://lastfm.freetls.fastly.net/i/u/300x300/...,<NA>
4,Bry,107494,https://www.last.fm/music/Bry/BRY,BRY,5651,"[rock, indie, indie rock, irish, ireland]",https://lastfm.freetls.fastly.net/i/u/300x300/...,<NA>
5,Gringe,371954,https://www.last.fm/music/Gringe/Enfant+lune,Enfant lune,14015,"[rap, france, french rap]",https://lastfm.freetls.fastly.net/i/u/300x300/...,<NA>
6,Pixies,15549432,https://www.last.fm/music/Pixies/Surfer+Rosa,Surfer Rosa,967234,"[alternative rock, 1988, indie rock, alternati...",https://lastfm.freetls.fastly.net/i/u/300x300/...,"07 Jul 2022, 16:03"
7,Liana Flores,4888773,https://www.last.fm/music/Liana+Flores/recently,recently,299013,"[comforting, 4, liana flores, acoustic, indie ...",https://lastfm.freetls.fastly.net/i/u/300x300/...,<NA>
8,Don Omar,985682,https://www.last.fm/music/Don+Omar/The+Last+Don,The Last Don,169392,"[reggaeton, don omar, latin, bachata, dance]",https://lastfm.freetls.fastly.net/i/u/300x300/...,<NA>
9,Foster the People,101139,https://www.last.fm/music/Foster+the+People/Th...,The Things We Do,27799,"[indie, disco, disco pop, indie pop, alternative]",https://lastfm.freetls.fastly.net/i/u/300x300/...,<NA>


In [15]:
albumInfo_csv.dtypes

artist_name    string
album_name     string
image          string
listeners       int32
playcount       int32
url            string
tag            object
content        string
track          object
published      string
dtype: object

In [46]:
albumInfo_csv.notnull().sum()

artist_name    66
album_name     66
image          66
listeners      66
playcount      66
url            66
tag            66
content        35
track          64
published      35
dtype: int64

## userinfo

In [43]:
params_user = {
    "method": "user.getInfo",
    "user": "",
    "api_key": "1a62f2d4937452d62d7426029e4c9997",
    "format": "json",
}

gender_dict = {'n' : 0}
userInfo_dataframe_list = []
for user, i in tqdm(user2id.items()):
    params_user['user'] = user
    user = requests.get(url, params_user).text
    user = json.loads(user)['user']
    user['image_url'] = user['image'][-1]['#text']
    user['registered_uts'] = user['registered']['unixtime']
    # print(user['gender'])
    user['gender'] = gender_dict[user['gender']]
    user = pd.DataFrame([user])
    user['follower'] = np.nan
    user['following'] = np.nan

    user.drop(columns=['playlists', 'image', 'type', 'artist_count', 'track_count', 'album_count', 'image', 'registered'], inplace=True)
    userInfo_dataframe_list.append(user)

userInfo_csv = pd.concat(userInfo_dataframe_list, ignore_index=True)
userInfo_csv.replace('None', np.NaN, inplace=True)
userInfo_csv.replace('', np.NaN, inplace=True)
userInfo_csv = userInfo_csv.astype({
    'name':'string',
    'age':'int',
    'subscriber':'int',
    'realname':'string',
    'bootstrap':'int',
    'playcount':'int',
    'image_url':'string',
    'realname':'string',
    'registered_uts':'string', # ?
    'country':'string',
    'gender':'int8',
    'url':'string',
})
display(userInfo_csv)
userInfo_csv.to_csv("userInfo_10.csv", index=False)

100%|██████████| 10/10 [00:02<00:00,  3.90it/s]


,name,age,subscriber,realname,bootstrap,playcount,country,gender,url,image_url,registered_uts,follower,following
0,cynocauma,0,0,<NA>,0,30021,<NA>,0,https://www.last.fm/user/cynocauma,<NA>,1645178226,NaN,NaN
1,AetherJoako,0,0,Joaquín,0,7142,Chile,0,https://www.last.fm/user/AetherJoako,https://lastfm.freetls.fastly.net/i/u/300x300/...,1541372926,NaN,NaN
2,leyrey,0,0,leyrey,0,118430,<NA>,0,https://www.last.fm/user/leyrey,https://lastfm.freetls.fastly.net/i/u/300x300/...,1540131248,NaN,NaN
3,NovicePlayer,0,0,<NA>,0,22942,<NA>,0,https://www.last.fm/user/NovicePlayer,<NA>,1653624752,NaN,NaN
4,Nouwh,0,0,<NA>,0,51038,<NA>,0,https://www.last.fm/user/Nouwh,https://lastfm.freetls.fastly.net/i/u/300x300/...,1571770571,NaN,NaN
5,erickdsr12,0,0,erick,0,217321,Brazil,0,https://www.last.fm/user/erickdsr12,https://lastfm.freetls.fastly.net/i/u/300x300/...,1427065817,NaN,NaN
6,ZombieTeam67,0,0,<NA>,0,83732,<NA>,0,https://www.last.fm/user/ZombieTeam67,https://lastfm.freetls.fastly.net/i/u/300x300/...,1637281617,NaN,NaN
7,BilingualZebra7,0,0,<NA>,0,20626,<NA>,0,https://www.last.fm/user/BilingualZebra7,<NA>,1641992153,NaN,NaN
8,rataclans,0,0,<NA>,0,17466,<NA>,0,https://www.last.fm/user/rataclans,<NA>,1645507377,NaN,NaN
9,Spectrum321146,0,0,<NA>,0,59185,<NA>,0,https://www.last.fm/user/Spectrum321146,https://lastfm.freetls.fastly.net/i/u/300x300/...,1597755411,NaN,NaN


In [65]:
userInfo_csv.dtypes

name            string
age              int64
subscriber       int64
realname        string
bootstrap        int64
playcount        int64
artist_count     int64
playlists        int64
track_count      int64
album_count      int64
image           string
registered      string
country         string
gender          string
url             string
type            string
dtype: object

## taginfo

In [51]:
len(tag2id)

16

In [45]:
params_tag = {
    "method": "tag.getInfo",
    "tag": "",
    "api_key": "1a62f2d4937452d62d7426029e4c9997",
    "format": "json",
}

print(tag2id)

tagInfo_dataframe_list = []
for tag, i in tqdm(tag2id.items()):
    params_tag['tag'] = tag
    tag = requests.get(url, params_tag).text
    tag = json.loads(tag)['tag']
    # print(tag.keys())
    # tag['summary'] = tag['wiki']['summary']
    # tag['content'] = tag['wiki']['content']
    tag = pd.DataFrame([tag])
    tag.drop(columns=['wiki'], inplace=True)
    
    tagInfo_dataframe_list.append(tag)

tagInfo_csv = pd.concat(tagInfo_dataframe_list, ignore_index=True)
tagInfo_csv.replace('', np.NaN, inplace=True)
tagInfo_csv = tagInfo_csv.astype({
    'name':'string',
    'total':'int',
    'reach':'int',
})
display(tagInfo_csv)
tagInfo_csv.to_csv("tagInfo.csv", index=False)

{'Hip-Hop': 0, 'rap': 1, 'conscious hip-hop': 2, 'trap': 3, 'Suomirap': 4, '2010s': 5, '2019': 6, 'pop': 7}


100%|██████████| 8/8 [00:01<00:00,  5.22it/s]


,name,total,reach
0,Hip-Hop,891992,129238
1,rap,595768,107037
2,conscious hip-hop,2723,566
3,trap,50140,9260
4,Suomirap,6371,349
5,2010s,53658,1290
6,2019,19725,644
7,pop,2023179,229787


In [18]:
display(tagInfo_csv.dtypes)
tagInfo_csv = tagInfo_csv.astype({
    'name':'string',
    'total':'int',
    'reach':'int',
    'summary':'string',
    'content':'string',
})
display(tagInfo_csv.head())
display(tagInfo_csv.dtypes)

name       string
total       int64
reach       int64
summary    string
content    string
dtype: object

,name,total,reach,summary,content
0,Suomirap,6371,349,"Rap from Finland, also check out Suomi Rock. <...","Rap from Finland, also check out Suomi Rock. <..."
1,2010s,53656,1290,"Music released from 2010 to the end of 2019, a...","Music released from 2010 to the end of 2019, a..."
2,2019,19725,644,"Music released in the year 2019, and non-2019 ...","Music released in the year 2019, and non-2019 ..."
3,Hip-Hop,891974,129236,"Hip-Hop, also spelled Hip Hop or HipHop is a r...","Hip-Hop, also spelled Hip Hop or HipHop is a r..."
4,pop,2023130,229785,Pop music is a broad-term for many different t...,Pop music is a broad-term for many different t...


name       string
total       int64
reach       int64
summary    string
content    string
dtype: object